In [3]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from time import sleep

textfile = open('files/capstonebeerstatebreweries.txt','r', encoding="utf-8")
filetext = textfile.read()
textfile.close()
matches = re.findall("/beer[a-zA-Z0-9/]+/", filetext)

r = requests.get("https://www.beeradvocate.com/articles/18442/top-50-most-rated-breweries-on-beeradvocate-for-2020/")
soup = bs(r.content, "html.parser")

beer_list = soup.select("ol li")
beer_count = 0
brewery_urls = []
for beer in beer_list:
    #print(beer.get_text().split(',')[0])
    link = beer.find("a")
    brewery_urls.append(link['href'])


def beer_lists(breweries):
    tap_list = []
    type_list = []
    abv_list = []
    ratings_count_list = []
    avg_rating_list = []
    brewery_list = []
    
    for brewery in breweries:
        rr = requests.get(f"https://www.beeradvocate.com{brewery}")
        soup_2 = bs(rr.content, "html.parser")
        new_list = soup_2.select("tbody tr a")
        non_link_list = soup_2.select("tbody tr td")
        brewery_name = soup_2.find("h1").get_text()
        #print(brewery_name)
        
        for beer_type in range(0,len(new_list),2):
            brewery_list.append(f"{brewery_name}")
            tap_list.append(f"{new_list[beer_type].get_text()}")
            type_list.append(f"{new_list[beer_type+1].get_text()}")
        
        for beer in range(0,len(non_link_list), 6 ):
            abv_entry = non_link_list[beer+2].get_text()
            if ',' in abv_entry:
                abv_entry = abv_entry.replace(',','')
            abv_entry = float(abv_entry)
            
            
            ratings_count_entry = non_link_list[beer+3].get_text()
            if ',' in ratings_count_entry:
                ratings_count_entry = ratings_count_entry.replace(',','')
            ratings_count_entry = int(ratings_count_entry)
            
            
            avg_rating_entry = non_link_list[beer+4].get_text()
            if ',' in avg_rating_entry:
                avg_rating_entry = avg_rating_entry.replace(',','')
            avg_rating_entry = float(avg_rating_entry)
            
            
            abv_list.append(f"{abv_entry}")
            ratings_count_list.append(f"{ratings_count_entry}")
            avg_rating_list.append(f"{avg_rating_entry}")
        

        #trying to do the archived lists
        rr = requests.get(f"https://www.beeradvocate.com{brewery}?view=beers&show=arc")
        soup_2 = bs(rr.content, "html.parser")
        new_list = soup_2.select("tbody tr a")
        non_link_list = soup_2.select("tbody tr td")

        for beer_type in range(0,len(new_list),2):
            brewery_list.append(f"{brewery_name}")
            tap_list.append(f"{new_list[beer_type].get_text()}")
            type_list.append(f"{new_list[beer_type+1].get_text()}")
        
        for beer in range(0,len(non_link_list), 6 ):
            abv_entry = non_link_list[beer+2].get_text()
            if ',' in abv_entry:
                abv_entry = abv_entry.replace(',','')
            abv_entry = float(abv_entry)
            
            
            ratings_count_entry = non_link_list[beer+3].get_text()
            if ',' in ratings_count_entry:
                ratings_count_entry = ratings_count_entry.replace(',','')
            ratings_count_entry = int(ratings_count_entry)
            
            
            avg_rating_entry = non_link_list[beer+4].get_text()
            if ',' in avg_rating_entry:
                avg_rating_entry = avg_rating_entry.replace(',','')
            avg_rating_entry = float(avg_rating_entry)
            
            
            abv_list.append(f"{abv_entry}")
            ratings_count_list.append(f"{ratings_count_entry}")
            avg_rating_list.append(f"{avg_rating_entry}")
        #print(len(tap_list))
        #sleep(0.2)
    return [brewery_list, tap_list,type_list, abv_list, ratings_count_list, avg_rating_list]

beers_and_types = beer_lists(matches)

brewery_list = beers_and_types[0]
beers = beers_and_types[1]
beer_types = beers_and_types[2]
abv = beers_and_types[3]
ratings_count = beers_and_types[4]
avg_rating = beers_and_types[5]

df_20 = pd.DataFrame(list(zip(brewery_list, beers, beer_types, abv, ratings_count, avg_rating)), columns=['Brewery Name','Beer Name', 'Beer Type', 'ABV %','Num of Ratings', 'Avg Rating'])

df_20.to_csv('files/top_20_state.csv')
df_20

,Brewery Name,Beer Name,Beer Type,ABV %,Num of Ratings,Avg Rating
0,Trimtab Brewing Company,205 Pale Ale,Pale Ale - American,5.5,25,4.0
1,Trimtab Brewing Company,205 SMaSH: Nelson,Pale Ale - American,5.5,1,4.34
2,Trimtab Brewing Company,5th Season,Blonde Ale - American,4.5,1,3.46
3,Trimtab Brewing Company,All Together,IPA - American,6.5,3,3.97
4,Trimtab Brewing Company,Angry Totem,Sour - Fruited Kettle Sour,7.5,2,4.22
...,...,...,...,...,...,...
187942,Library Sports Grille & Brewery,O'Dwyer's Irish,Red Ale - Irish,0.0,0,0.0
187943,Library Sports Grille & Brewery,ODB,Wheat Beer - American Pale,0.0,0,0.0
187944,Library Sports Grille & Brewery,Red Eye,Red Ale - American Amber / Red,0.0,0,0.0
187945,Library Sports Grille & Brewery,Sigmund's Nut Brown,Brown Ale - English,0.0,0,0.0


In [3]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from time import sleep
df_2 = pd.read_csv('files/top_20_state.csv')

df_2['ABV %'] = pd.to_numeric(df_2['ABV %'])
df_2['Num of Ratings'] = pd.to_numeric(df_2['Num of Ratings'])
df_2['Avg Rating'] = pd.to_numeric(df_2['Avg Rating'])
#gives us the df with all pertintent data - beer names (which we can sort by beer type, ABV average, Num of Ratings, and Avg rating)

In [4]:
avg_grouped_ratings20 = df_2.groupby('Beer Type', as_index=False).mean()['Avg Rating']
#insert this into new column
avg_grouped_ratings20

avg_grouped_abv20 = df_2.groupby('Beer Type', as_index=False).mean()['ABV %']
#insert this into new column
avg_grouped_abv20

grouped_df20 = df_2.groupby('Beer Type', as_index=False).sum()[['Beer Type','Num of Ratings']]
grouped_df20['Avg of All Ratings'] = round(avg_grouped_ratings20,2)
grouped_df20['Avg ABV %'] = round(avg_grouped_abv20,2)

In [6]:
df_2['Num of Ratings'].sum()

5597352

#### Testing State Classification

In [4]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from time import sleep

textfile = open('files/capstonebeerstatebreweries.txt','r', encoding="utf-8")
filetext = textfile.read()
textfile.close()
matches = re.findall("/beer[a-zA-Z0-9/]+/", filetext)

r = requests.get("https://www.beeradvocate.com/articles/18442/top-50-most-rated-breweries-on-beeradvocate-for-2020/")
soup = bs(r.content, "html.parser")

beer_list = soup.select("ol li")
beer_count = 0
brewery_urls = []
for beer in beer_list:
    #print(beer.get_text().split(',')[0])
    link = beer.find("a")
    brewery_urls.append(link['href'])


def beer_lists(breweries):
    tap_list = []
    type_list = []
    abv_list = []
    ratings_count_list = []
    avg_rating_list = []
    brewery_list = []
    city_list = []
    state_list = []
    
    for brewery in breweries:
        rr = requests.get(f"https://www.beeradvocate.com{brewery}")
        soup_2 = bs(rr.content, "html.parser")
        new_list = soup_2.select("tbody tr a")
        non_link_list = soup_2.select("tbody tr td")
        brewery_name = soup_2.find("h1").get_text()
        brew_city = brew_state = soup_2.select("#info_box a")[0].get_text()
        brew_state = soup_2.select("#info_box a")[1].get_text()
        #testing locating state
        print(brewery_name)


        
        for beer_type in range(0,len(new_list),2):
            state_list.append(brew_state)
            city_list.append(brew_city)
            brewery_list.append(f"{brewery_name}")
            tap_list.append(f"{new_list[beer_type].get_text()}")
            type_list.append(f"{new_list[beer_type+1].get_text()}")
        
        for beer in range(0,len(non_link_list), 6 ):
            abv_entry = non_link_list[beer+2].get_text()
            if ',' in abv_entry:
                abv_entry = abv_entry.replace(',','')
            abv_entry = float(abv_entry)
            
            
            ratings_count_entry = non_link_list[beer+3].get_text()
            if ',' in ratings_count_entry:
                ratings_count_entry = ratings_count_entry.replace(',','')
            ratings_count_entry = int(ratings_count_entry)
            
            
            avg_rating_entry = non_link_list[beer+4].get_text()
            if ',' in avg_rating_entry:
                avg_rating_entry = avg_rating_entry.replace(',','')
            avg_rating_entry = float(avg_rating_entry)
            
            
            abv_list.append(f"{abv_entry}")
            ratings_count_list.append(f"{ratings_count_entry}")
            avg_rating_list.append(f"{avg_rating_entry}")
        

        #trying to do the archived lists
        rr = requests.get(f"https://www.beeradvocate.com{brewery}?view=beers&show=arc")
        soup_2 = bs(rr.content, "html.parser")
        new_list = soup_2.select("tbody tr a")
        non_link_list = soup_2.select("tbody tr td")

        for beer_type in range(0,len(new_list),2):
            city_list.append(brew_city)
            state_list.append(brew_state)
            brewery_list.append(f"{brewery_name}")
            tap_list.append(f"{new_list[beer_type].get_text()}")
            type_list.append(f"{new_list[beer_type+1].get_text()}")
        
        for beer in range(0,len(non_link_list), 6 ):
            abv_entry = non_link_list[beer+2].get_text()
            if ',' in abv_entry:
                abv_entry = abv_entry.replace(',','')
            abv_entry = float(abv_entry)
            
            
            ratings_count_entry = non_link_list[beer+3].get_text()
            if ',' in ratings_count_entry:
                ratings_count_entry = ratings_count_entry.replace(',','')
            ratings_count_entry = int(ratings_count_entry)
            
            
            avg_rating_entry = non_link_list[beer+4].get_text()
            if ',' in avg_rating_entry:
                avg_rating_entry = avg_rating_entry.replace(',','')
            avg_rating_entry = float(avg_rating_entry)
            
            
            abv_list.append(f"{abv_entry}")
            ratings_count_list.append(f"{ratings_count_entry}")
            avg_rating_list.append(f"{avg_rating_entry}")
        print(brew_city)
        print(brew_state)    
        print(len(tap_list))
        
        sleep(0.5)
    print(len(brewery_list))
    return [brewery_list, tap_list,type_list, abv_list, ratings_count_list, avg_rating_list, city_list, state_list]


print(len(matches))


beers_and_types = beer_lists(matches)

brewery_list = beers_and_types[0]
beers = beers_and_types[1]
beer_types = beers_and_types[2]
abv = beers_and_types[3]
ratings_count = beers_and_types[4]
avg_rating = beers_and_types[5]
cities = beers_and_types[6]
states = beers_and_types[7]

df_20 = pd.DataFrame(list(zip(brewery_list, cities, states, beers, beer_types, abv, ratings_count, avg_rating)), columns=['Brewery Name','City','State','Beer Name', 'Beer Type', 'ABV %','Num of Ratings', 'Avg Rating'])

df_20.to_csv('files/top20_state_with_cities_test.csv')
df_20


1012
Trimtab Brewing Company
Birmingham
Alabama
163
Straight To Ale
Huntsville
Alabama
295
Fairhope Brewing Company
Fairhope
Alabama
396
Yellowhammer Brewing
Huntsville
Alabama
482
Good People Brewing Company
Birmingham
Alabama
570
Rocket Republic Brewing Company
Madison
Alabama
620
Back Forty Beer Co.
Gadsden
Alabama
668
Avondale Brewing Co.
Birmingham
Alabama
731
Singin' River Brewing Co.
Florence
Alabama
778
Cahaba Brewing Company
Birmingham
Alabama
844
Below the Radar Brewhouse
Huntsville
Alabama
874
Green Bus Brewing
Huntsville
Alabama
905
Cross-Eyed Owl Brewing Company
Decatur
Alabama
935
Mad Malts
Huntsville
Alabama
964
Ghost Train Brewing Company
Birmingham
Alabama
1000
Birmingham District Brewing
Birmingham
Alabama
1028
Serda Brewing Company
Mobile
Alabama
1055
Salty Nut Brewery
Huntsville
Alabama
1090
Tallulah Brewing Co.
Jasper
Alabama
1116
Fractal Brewing Project
Huntsville
Alabama
1140
Anchorage Brewing Company
Anchorage
Alaska
1334
49th State Brewing Company
Healy
Alaska


Hellbender Brewing Company
Washington
District of Columbia
36656
DC Brau Brewing Co.
Washington
District of Columbia
36821
Atlas Brew Works
Washington
District of Columbia
36868
Bardo Brewpub
Washington
District of Columbia
36900
Red Bear Brewing Company
Washington
District of Columbia
36930
District ChopHouse & Brewery
Washington
District of Columbia
36972
The Public Option
Washington
District of Columbia
36997
Valor Brewpub
Washington
District of Columbia
37005
Pulaski True American
Washington
District of Columbia
37011
888 Lucky Beer
Washington
District of Columbia
37015
Sankofa Beer Co.
Washington
District of Columbia
37018
Other Half Brewing Company DC
Washington
District of Columbia
37020
Soul Mega
Washington
District of Columbia
37021
J. Wakefield Brewing
Miami
Florida
37678
Cigar City Brewing
Tampa
Florida
38832
Angry Chair Brewing
Tampa
Florida
39191
Funky Buddha Brewery
Oakland Park
Florida
39632
Hourglass Brewing
Longwood
Florida
39861
Tripping Animals Brewing
Doral
Florida


Braxton Brewing Company
Covington
Kentucky
69217
West Sixth Brewing Company
Lexington
Kentucky
69448
Ethereal Brewing
Lexington
Kentucky
69599
3rd Turn Brewing
Jefferstown
Kentucky
69714
Country Boy Brewing
Lexington
Kentucky
69904
Mile Wide Beer Co.
Louisville
Kentucky
69976
Blue Stallion Brewing Co.
Lexington
Kentucky
70073
Great Flood Brewing Company
Louisville
Kentucky
70124
Bircus Brewing Company
Ludlow
Kentucky
70168
Falls City Brewing Company
Louisville
Kentucky
70229
Alexandria Brewing Company
Alexandria
Kentucky
70272
Henderson Brewing Company
Henderson
Kentucky
70321
Dreaming Creek Brewery
Richmond
Kentucky
70367
Wooden Cask Brewing Company
Newport
Kentucky
70403
Akasha Brewing Company
Louisville
Kentucky
70440
Gravely Brewing Co.
Louisville
Kentucky
70475
Darkness Brewing
Bellevue
Kentucky
70512
Monnik Beer Co.
Louisville
Kentucky
70564
Urban South Brewery
New Orleans
Louisiana
70926
New Orleans Lager & Ale Brewing Company (NOLA)
New Orleans
Louisiana
71152
The Courtyard Bre

Maplewood
Missouri
100897
Urban Chestnut Brewing Company - Midtown Brewery & Biergarten
Saint Louis
Missouri
101027
Martin City Brewing Company
Kansas City
Missouri
101125
Mother's Brewing Company
Springfield
Missouri
101241
Broadway Brewery & Restaurant
Columbia
Missouri
101348
Crane Brewing Company
Raytown
Missouri
101437
Heavy Riff Brewing Company
Saint Louis
Missouri
101519
Alma Mader Brewing
Kansas City
Missouri
101590
Blackfoot River Brewing Company
Helena
Montana
101666
Bozeman Brewing Company
Bozeman
Montana
101730
Überbrew
Billings
Montana
101800
Mountains Walking Brewery
Bozeman
Montana
101856
Big Sky Brewing Company
Missoula
Montana
101941
Imagine Nation Brewing Co.
MIssoula
Montana
101995
KettleHouse Brewing Co. (Southside Taproom)
Missoula
Montana
102042
KettleHouse Brewing Co. - Northside Taproom
Missoula
Montana
102089
KettleHouse Brewing Co
Bonner
Montana
102126
Red Lodge Ales
Red Lodge
Montana
102175
Carter's Brewing
Billings
Montana
102234
Draught Works
Missoula
Monta

Concord
North Carolina
127325
Green Man Brewery
Asheville
North Carolina
127522
Sycamore Brewing
Charlotte
North Carolina
127662
New Anthem Beer Project
Wilmington
North Carolina
127784
HopFly Brewing Co.
Rocky Mount
North Carolina
127900
Newgrass Brewing Company
Shelby
North Carolina
128019
Drekker Brewing Company
Fargo
North Dakota
128250
Souris River Brewing
Minot
North Dakota
128324
Fargo Brewing Company
Fargo
North Dakota
128403
Granite City Food & Brewery
Fargo
North Dakota
128482
Half Brothers Brewing Company
Grand Forks
North Dakota
128532
Rhombus Guys Brewing Company
Grand Forks
North Dakota
128576
Laughing Sun Brewing Company
Bismarck
North Dakota
128618
Drumconrath Brewing Company
Fargo
North Dakota
128650
Stonehome Brewing Company
Watford City
North Dakota
128673
Prairie Brothers Brewery and Taproom
Fargo
North Dakota
128693
Bismarck Brewing
Bismarck
North Dakota
128713
Phat Fish Brewing Co.
Dickinson
North Dakota
128731
Bird Dog Brewing
Bismarck
North Dakota
128748
Buffalo

Dakota Point Brewing
Rapid City
South Dakota
157865
Fernson Brewing Company
Sioux Falls
South Dakota
157960
Eponymous Brewing Co
Brookings
South Dakota
158014
Granite City Food & Brewery
Sioux Falls
South Dakota
158093
Spearfish Brewing Company
Spearfish
South Dakota
158151
Covert Artisan Ales
Sioux Falls
South Dakota
158207
Zymurcracy Beer Company
Rapid City
South Dakota
158256
Ben's Brewing Co.
Yankton
South Dakota
158304
The Knuckle Saloon & Grill
Sturgis
South Dakota
158351
Hay Camp Brewing Company
Rapid City
South Dakota
158400
Southern Grist Brewing Co.
Nashville
Tennessee
158860
Bearded Iris Brewing
Nashville
Tennessee
159095
Yazoo Brewing Company
Madison
Tennessee
159394
TailGate Brewery
Nashville
Tennessee
159523
Blackberry Farm Brewery
Maryville
Tennessee
159636
Wiseacre Brewing
Memphis
Tennessee
159737
Pretentious Beer Co.
Knoxville
Tennessee
159824
Mantra Artisan Ales
Franklin
Tennessee
159889
Smith & Lentz Brewing Company
Nashville
Tennessee
159950
New Heights Brewery
Nash

Milwaukee Brewing Company
Milwaukee
Wisconsin
186685
Minhas Craft Brewery
Monroe
Wisconsin
186818
Next Door Brewing Company
Madison
Wisconsin
186900
Titletown Brewing Company
Green Bay
Wisconsin
186995
Brewfinity Brewing Company
Oconomowoc
Wisconsin
187064
Working Draft Beer Co.
Madison
Wisconsin
187126
Melvin Brewing / Thai Me Up
Jackson
Wyoming
187291
Snake River Brewing Company & Brewpub
Jackson
Wyoming
187431
Black Tooth Brewing Company
Sheridan
Wyoming
187498
Luminous Brewhouse
Sheridan
Wyoming
187552
Altitude Chophouse & Brewery
Laramie
Wyoming
187616
Frontier Brewing Company and Taproom
Casper
Wyoming
187663
Freedom's Edge Brewing Co.
Cheyenne
Wyoming
187711
Coal Creek TAP
Laramie
Wyoming
187760
Lander Brewing Company
Lander
Wyoming
187814
Accomplice Beer Company
Cheyenne
Wyoming
187857
Roadhouse Brewing Co.
Jackson
Wyoming
187910
Wind River Brewing Company
Pinedale
Wyoming
187948
Suds Brothers Brewery
Evanston
Wyoming
187981
Gillette Brewing Company
Gillette
Wyoming
188013
One 

,Brewery Name,City,State,Beer Name,Beer Type,ABV %,Num of Ratings,Avg Rating
0,Trimtab Brewing Company,Birmingham,Alabama,205 Pale Ale,Pale Ale - American,5.5,25,4.0
1,Trimtab Brewing Company,Birmingham,Alabama,205 SMaSH: Nelson,Pale Ale - American,5.5,1,4.34
2,Trimtab Brewing Company,Birmingham,Alabama,5th Season,Blonde Ale - American,4.5,1,3.46
3,Trimtab Brewing Company,Birmingham,Alabama,All Together,IPA - American,6.5,3,3.97
4,Trimtab Brewing Company,Birmingham,Alabama,Angry Totem,Sour - Fruited Kettle Sour,7.5,2,4.22
...,...,...,...,...,...,...,...,...
188180,Library Sports Grille & Brewery,Laramie,Wyoming,O'Dwyer's Irish,Red Ale - Irish,0.0,0,0.0
188181,Library Sports Grille & Brewery,Laramie,Wyoming,ODB,Wheat Beer - American Pale,0.0,0,0.0
188182,Library Sports Grille & Brewery,Laramie,Wyoming,Red Eye,Red Ale - American Amber / Red,0.0,0,0.0
188183,Library Sports Grille & Brewery,Laramie,Wyoming,Sigmund's Nut Brown,Brown Ale - English,0.0,0,0.0


In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from time import sleep

df2021 = pd.read_csv('files/beer_data2021.csv')



In [10]:
import numpy as np
df2021
df2021.groupby('State', as_index = False).sum()[['State','Num of Ratings']].sort_values('Num of Ratings', ascending=False)

,State,Num of Ratings
21,Massachusetts,493332
22,Michigan,426662
4,California,375564
37,Oregon,349884
7,Delaware,341712
33,North Carolina,296487
5,Colorado,276401
13,Illinois,255757
46,Virginia,251482
25,Missouri,235239


In [26]:
df2012 = pd.read_csv('files/beer_data2012.csv', usecols=['brewery_name','beer_name','beer_style','beer_abv','review_overall'])
df2021['Count'] = 1


df2021.groupby('Beer Type')

#Breakdown of number of each beer type (unsorted)
beertype_count2021 = df2021.groupby('Beer Type').count()[['Count']]

#Breweries ranked by number of unique beers
beercount2021_by_brewery = df2021.groupby('Brewery Name', as_index=False).count()[['Brewery Name','Count']]
beercount2021_by_brewery.sort_values('Count', ascending=False)
#It became evident here that there were several locations (Rock Bottom Restaurant, Iron Hill Brewery & Restaurant, Pizza Port) that did not list their unique beers
#Applied quick filter to remove those from the listings
beercount2021_by_brewery = beercount2021_by_brewery[(~beercount2021_by_brewery['Brewery Name'].str.contains("Rock Bottom Restaurant")) & (~beercount2021_by_brewery['Brewery Name'].str.contains("Iron Hill Brewery & Restaurant")) & (~beercount2021_by_brewery['Brewery Name'].str.contains("Pizza Port"))].sort_values('Count', ascending=False).head(25)
beercount2021_by_brewery

#Number of ratings in total for each beer type (unsorted)
beertype2021_sumreviews = df2021.groupby('Beer Type').sum()[['Num of Ratings']]

#Avg review rating for each beertype (unsorted)
beertype2021_avgreviews = round(df2021.groupby('Beer Type').mean()[['Avg Rating']],2)



#Average rating for all beers belonging to each brewery - then grouped by brewery name and state location
topbreweries_reviews_2021 = round(df2021.groupby(['Brewery Name','State']).mean()[['Avg Rating']],3)
#Sorted by avg rating and narrowed down to top 100
top100breweries_2021 = topbreweries_reviews_2021.sort_values('Avg Rating', ascending=False).head(100)

'''
Creating ABV bins for further analysis
Also filtering out beers that have an ABV of 0.0%, as while they have valid reviews, their ABV is not correctly listed
As this results their ABV average to come back as NaN
'''
#Defining ranges of bins and giving them names/labels
bins = [0,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5, 10.5, np.inf]
names = ['<2.5%','2.6-3.5%','3.6-4.5%','4.6-5.5%','5.6-6.5%','6.6-7.5%','7.6-8.5%','8.6-9.5%','9.6-10.5%','10.6%+']

#Adding in bin classification for each row/item in the df
df2021['ABV Range'] = pd.cut(df2021['ABV %'], bins, labels=names)

#Filtering out an NaN returns, as there was a number of beers that came back with 0.0% ABV or other values that throw off mean()
df2021_abvranges = df2021[df2021['ABV Range'].notnull()]

df2021_abvranges = df2021_abvranges.groupby('ABV Range').count().sort_values('Count', ascending=False)['Count']
df2021_abvratings = round(df2021.groupby('ABV Range').mean().sort_values('Avg Rating', ascending=False),2)['Avg Rating']
df2021_abvranges
#Number of beers that fall into each ABV range
df2021_abvranges
#Average review score of each ABV range
df2021_abvratings


state_review_count = df2021.groupby('State', as_index=False).sum().sort_values('Num of Ratings', ascending=False)[['State','Num of Ratings']]
state_review_count


#Ranking states by average brewery review score
states_ranked_2021 = round(df2021.groupby(['Brewery Name','State'], as_index=False).mean()[['State','Avg Rating']],3)
states_ranked_2021 = states_ranked_2021.groupby('State', as_index=False).mean().sort_values('Avg Rating', ascending=False)


#Top 25 Breweries ranked by number of unique beers
beercount2021_by_brewery = df2021.groupby('Brewery Name', as_index=False).count()[['Brewery Name','Count']]
beercount2021_by_brewery.sort_values('Count', ascending=False)
#It became evident here that there were several locations (Rock Bottom Restaurant, Iron Hill Brewery & Restaurant, Pizza Port) that did not list their unique beers
#Applied quick filter to remove those from the listings
beercount2021_by_brewery = beercount2021_by_brewery[(~beercount2021_by_brewery['Brewery Name'].str.contains("Rock Bottom Restaurant")) & (~beercount2021_by_brewery['Brewery Name'].str.contains("Iron Hill Brewery & Restaurant")) & (~beercount2021_by_brewery['Brewery Name'].str.contains("Pizza Port"))].sort_values('Count', ascending=False).head(25)
beercount2021_by_brewery

#Top 25 breweries ranked by number of reviews/ratings of their beers combined
top25_breweries_reviewcount_2021 = df2021.groupby('Brewery Name', as_index=False).sum()[['Brewery Name','Num of Ratings']]
#It became evident here that there were several locations (Rock Bottom Restaurant, Iron Hill Brewery & Restaurant, Pizza Port) that did not list their unique beers
#Applied quick filter to remove those from the listings
top25_breweries_reviewcount_2021 = top25_breweries_reviewcount_2021[(~top25_breweries_reviewcount_2021['Brewery Name'].str.contains("Rock Bottom Restaurant")) & (~top25_breweries_reviewcount_2021['Brewery Name'].str.contains("Iron Hill Brewery & Restaurant")) & (~top25_breweries_reviewcount_2021['Brewery Name'].str.contains("Pizza Port"))].sort_values('Num of Ratings', ascending=False).head(25)




#writing all 2021 data to CSV
beertype_count2021.to_csv('finished_data/2021/beertype_count2021.csv')
beercount2021_by_brewery.to_csv('finished_data/2021/beercount_by_brewery2021.csv')
beertype2021_sumreviews.to_csv('finished_data/2021/beertype_sumreview2021.csv')
beertype2021_avgreviews.to_csv('finished_data/2021/beertype_avgreviews2021.csv')
topbreweries_reviews_2021.to_csv('finished_data/2021/topbreweries_reviews2021.csv')
top100breweries_2021.to_csv('finished_data/2021/top100_breweries2021.csv')
top25_breweries_reviewcount_2021.to_csv('finished_data/2021/top25_breweries_byreviewcount_2021.csv')


state_review_count.to_csv('finished_data/2021/state_review_count2021.csv')
states_ranked_2021.to_csv('finished_data/2021/states_ranked_2021.csv')

df2021_abvratings.to_csv('finished_data/2021/abv_range_avgreview2021.csv')
df2021_abvranges.to_csv('finished_data/2021/abv_range_reviewcount2021.csv')

In [36]:
df2021
topbreweries_reviews_2021 = round(df2021.groupby(['Brewery Name','City','State']).mean()[['Avg Rating']],3)
#topbreweries_reviews_2021.to_csv('finished_data/2021/topbreweries_reviews2021.csv')

topbreweries_reviews_2021.sort_values('Avg Rating', ascending=False)

,,,Avg Rating
Brewery Name,City,State,
Troon Brewing,Hopewell,New Jersey,4.296
Hill Farmstead Brewery,Greensboro Bend,Vermont,4.255
Tree House Brewing Company,Charlton,Massachusetts,4.251
Trillium Brewing Company,Boston,Massachusetts,4.206
River Roost Brewery,White River Junction,Vermont,4.206
...,...,...,...
Valor Brewpub,Washington,District of Columbia,0.000
Pulaski True American,Washington,District of Columbia,0.000
Mayhew Junction Brewing Company,Starkville,Mississippi,0.000


In [29]:
df2012 = df2012[['brewery_name', 'beer_name', 'beer_style', 'beer_abv', 'review_overall']]
df2012['Count'] = 1
df2012.groupby('beer_style').count()[['Count']]

,Count
beer_style,
Altbier,7741
American Adjunct Lager,30749
American Amber / Red Ale,45751
American Amber / Red Lager,9311
American Barleywine,26728
...,...
Vienna Lager,8954
Weizenbock,9412
Wheatwine,3714


In [30]:
#Because of how this dataset is structured/gathered, it is not as flexible as the one I scraped/put together myself

#Leaving these as unsorted in an attempt to make it easier to merge all beertype-specific data into one df
#Number of each beer type, unsorted
beertype_count2012= df2012.groupby('beer_style').count()[['Count']]
beertype_count2012

#Avg of reviews for each beer type (grouped by beer type, unsorted)
beertype2012_avgreviews = round(df2012.groupby('beer_style').mean()[['review_overall']],2)
beertype2012_avgreviews

#Average rating for all beers belonging to each brewery
topbreweries_reviews_2012 = round(df2012.groupby('brewery_name').mean()[['review_overall']],3)
#Sorted by avg rating and narrowed down to top 100
top100breweries_2012 = topbreweries_reviews_2012.sort_values('brewery_name', ascending=True)
#After reviewing the data, it became apparent there were some breweries/locations with VERY small beer counts and beer reviews
#This led to skewed results of trying to rang via review-overall, as some places had 1-2 beers/ratings 
top100breweries_2012['Count'] = df2012.groupby('brewery_name').sum()[['Count']]
#After implementing a 'Count' column for # of beers at each brewery and reviewing results, set threshold for ranking at 30 beers
top100breweries_2012 = top100breweries_2012[top100breweries_2012['Count']>= 50]

top100breweries_2012 = top100breweries_2012.sort_values('review_overall', ascending=False).head(100)
top100breweries_2012.head(20)

#Top 25 breweries for 2012 ranked by review count(since beer count is not unique in 2012 data, 1.59mil beers listed and 56.8k unique)
top25_breweries_reviewcount_2012 = df2012.groupby('brewery_name',as_index=False).count()[['brewery_name','Count']].sort_values('Count',ascending=False).head(25)


'''
Creating ABV bins for further analysis of 2012 Data as well
'''
#Defining ranges of bins and giving them names/labels
bins_2012 = [0,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5, 10.5, np.inf]
names_2012 = ['<2.5%','2.6-3.5%','3.6-4.5%','4.6-5.5%','5.6-6.5%','6.6-7.5%','7.6-8.5%','8.6-9.5%','9.6-10.5%','10.6%+']

#Adding in bin classification for each row/item in the df
df2012['ABV Range'] = pd.cut(df2012['beer_abv'], bins_2012, labels=names_2012)

#Filtering out an NaN returns, as there was a number of beers that came back with 0.0% ABV or other values that throw off mean()
df2012_abvranges = df2012[df2012['ABV Range'].notnull()]

df2012_abvranges = df2012_abvranges.groupby('ABV Range').count().sort_values('Count', ascending=False)['Count']
df2012_abvratings = round(df2012.groupby('ABV Range').mean().sort_values('review_overall', ascending=False),2)['review_overall']
#beer counts belonging to each range of ABV's
df2012_abvranges
#avg ratings for each ABV range
df2012_abvratings



#Writing all 2012 data to CSV
beertype_count2012.to_csv('finished_data/2012/beertype_count2012.csv')
beertype2012_avgreviews.to_csv('finished_data/2012/beertype_avgreviews2012.csv')
topbreweries_reviews_2012.to_csv('finished_data/2012/allbreweries_avg_reviews.csv')
top100breweries_2012.to_csv('finished_data/2012/top100breweries_avg_reviews.csv')
top25_breweries_reviewcount_2012.to_csv('finished_data/2012/top25_breweries_byreviewcount_2012.csv')

df2012_abvranges.to_csv('finished_data/2012/abv_range_count2021.csv')
df2012_abvratings.to_csv('finished_data/2012/abv_range_avgreview2012.csv')

In [53]:

top25_breweries_reviewcount_2012.to_csv('finished_data/2012/top25_breweries_byreviewcount_2012.csv')
top25_breweries_reviewcount_2021.to_csv('finished_data/2021/top25_breweries_byreviewcount_2021.csv')


In [83]:
df2021[df2021['State'] == 'Alaska'].groupby('Brewery Name')['Avg Rating'].mean()

3.216500356379186

In [103]:
states_ranked_2021 = round(df2021.groupby(['Brewery Name','State'], as_index=False).mean()[['State','Avg Rating']],3)
states_ranked_2021 = states_ranked_2021.groupby('State', as_index=False).mean().sort_values('Avg Rating', ascending=False)
states_ranked_2021.to_csv('finished_data/2021/states_ranked_2021.csv')

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'